In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [229]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi
from exact_sync.v1.api.products_api import ProductsApi
from exact_sync.v1.api.teams_api import TeamsApi

from exact_sync.v1.models import ImageSet, Team, Product, AnnotationType, Image, Annotation, AnnotationMediaFile
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

In [5]:
configuration = Configuration()
configuration.username = 'marzahl'
configuration.password = '***'
configuration.host =  "https://exact.cs.fau.de/"

image_set_ids = [246, 245, 251, 15, 222, 247]

In [4]:
client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)
product_api = ProductsApi(client)

In [6]:
images = {}
annotation_types = {}

for image_set in image_set_ids:
    for image in images_api.list_images(pagination=False, image_set=image_set).results:
        images[image.id] = image
    for product in product_api.list_products(imagesets=image_set).results:
        for anno_type in annotation_types_api.list_annotation_types(product=product.id).results:
            annotation_types[anno_type.id] = anno_type

In [12]:
def set_species(image_set_id):
    
    species = "equine"
    if image.image_set == 251:
        species = "human"
    if image.image_set == 222:
        species = "felis catus"
        
    return species

In [13]:
def data_set(image_set_id):
    
    data_set = "SREP"
    if image_set_id not in [245, 15]:
        data_set = "SDATA"
    return data_set

In [14]:
def set_version(data_set, deleted, last_editor):
    
    version = "SREP"
    
    if data_set == "SREP" and deleted == False:
        version = "SREP_V2"
        
    if data_set != "SREP":
        version = "Inference"
        
        if deleted:
            if last_editor == 1:
                version = "Cluster"
            else:
                version = "Screening"
        elif last_editor != 1 and deleted ==  False:
            version = "DensityMap"
    return version
            

In [188]:
rows = []

for annotation_type_id in [338, 337]:
    annotation_type_name = "Granulozyten" if  annotation_type_id == 337 else "Delete"
    for anno in annotations_api.list_annotations(annotation_type=annotation_type_id, pagination=False, fields="id,unique_identifier,annotation_type,vector,user,deleted,last_editor").results:
        
        
        
        species = "felis catus"
        data_set_name = "SDATA"
        version = "Cluster"
        image_type = "Turnbull"
        if anno.vector is not None: 
            rows.append([anno.id, anno.image, 249, species, image_type, annotation_type_name, anno.vector, anno.unique_identifier, anno.user, anno.deleted, anno.last_editor, data_set_name, version])
            
    dfCC = pd.DataFrame(rows, columns=['id', 'image_id', 'image_set', 'species' , 'image_type', 'name', 'vector', 'unique_identifier', 'user_id', 'deleted', 'last_editor', 'data_set_name', 'version'])
    dfCC.to_pickle("EIPH_ClusterFilter_Annotations.pkl")

In [16]:
rows = []
for image_id, image in tqdm(images.items()):

    for annotation_type_id, annotation_type in annotation_types.items():

        for anno in annotations_api.list_annotations(annotation_type=annotation_type_id, image=image_id, pagination=False, fields="id,unique_identifier,annotation_type,vector,user,deleted,last_editor").results:
                  
            species = set_species(image.image_set)
            data_set_name = data_set(image.image_set)
            version = set_version(data_set_name, anno.deleted, anno.last_editor)
            
            # anno_id, image_id, image_name, anno_type_name, vector, user_id, user_name
            image_type = "Prussian" if "erliner" in image.name else "Turnbull"
            if anno.vector is not None and anno.user in [1, 11, 12]:
                rows.append([anno.id, image_id, image.image_set, species, image.name, image_type, int(annotation_type.name), anno.vector, anno.unique_identifier, anno.user, anno.deleted, anno.last_editor, data_set_name, version])

    df = pd.DataFrame(rows, columns=['id', 'image_id', 'image_set', 'species' ,'image_name', 'image_type', 'grade', 'vector', 'unique_identifier', 'user_id', 'deleted', 'last_editor', 'data_set_name', 'version'])
    df.to_pickle("EIPH_Annotations.pkl")

100%|████████████████████████████████████████████████████████████████████████████████| 91/91 [2:04:27<00:00, 82.06s/it]
